In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score, confusion_matrix
import seaborn as sns
from random import sample, seed 
from scipy import stats

import matplotlib.pyplot as plt
from gseapy.plot import barplot, dotplot
from mycolorpy import colorlist as mcp
from matplotlib.lines import Line2D



In [45]:
#Read omics data
omics_prot_path = '/data/sharedData/UK_BIOBANK_DATA/Download_Data/ProteomicsData/DATAforMARTIN/Olink_proteomics_data_transposed_decoded2UNIportID_stringvalue_corrected_Martin.txt'
omics_met_path = '/data/sharedData/UK_BIOBANK_DATA/Download_Data/Metabolomics_Data/SecondPhaseData/nmr_biomarker_data_RemovedTechVariation_secondPHASE_FilteredDuplicateV2.csv'

omics_prot = pd.read_csv(omics_prot_path, sep = ',', index_col = 0)
omics_met = pd.read_csv(omics_met_path, sep = ',', index_col = 0)


# Set up the data formats for the output
boxplot_data_full = ({'prob':[],
                'reference':[],
                'disease':[],
                'data_type':[],
                'Model': []
                })
boxplot_data_full = pd.DataFrame(boxplot_data_full)

Shiny_data = ({'prob':[],
                'reference':[],
               'Disease':[],
               'Data': [],
               'N':[],
               'AUC_train':[],
               'AUC_test':[],
               'Features':[],
               'Coef':[],
               'Model':[]
              })
Shiny_data = pd.DataFrame(Shiny_data)


for disease in ['CD', 'UC', 'RA', 'PSO', 'SLE', 'COPD', 'obesity', 'T2D', 'Atherosclerosis']:
#for disease in ['CD']:
    for data_type in ['prot', 'met']:
        if data_type == 'prot':
            omics = omics_prot.copy()
        else:
            omics = omics_met.copy()
            
        for model_type in ['incident', 'prevalent']:           


            if model_type == 'incident':
                sick_path = '/home/marsm05/Documents/FBI_project/UKbb/clinical_story/ukb_' + \
                            disease + '_' + data_type + '_incident.csv'    
                clinical_sick = pd.read_csv(sick_path, index_col = 0)
                HC_path = '/home/marsm05/Documents/FBI_project/UKbb/clinical_story/ukb_' + \
                            disease + '_' + data_type + '_HC_PairedToIncident.csv'
                clinical_healthy = pd.read_csv(HC_path, index_col = 0)
            else:
                sick_path = '/home/marsm05/Documents/FBI_project/UKbb/clinical_story/ukb_' + \
                            disease + '_' + data_type + '_prevalent.csv'    
                clinical_sick = pd.read_csv(sick_path, index_col = 0)
                HC_path = '/home/marsm05/Documents/FBI_project/UKbb/clinical_story/ukb_' + \
                            disease + '_' + data_type + '_HC_PairedToPrevalent.csv'
                clinical_healthy = pd.read_csv(HC_path, index_col = 0)


            # clinical -> clinical data from both healthy and sick
            # omics_clinical -> subset of metabolomics/proteomics including only patients from clinical data
            clinical = pd.concat([clinical_sick,clinical_healthy])

            omics_clinical = omics.loc[clinical.index.values]

            #remove proteins/metabolites with more than 10% NA
            omics_clinical = omics_clinical[omics_clinical.columns[omics_clinical.isna().sum() < np.shape(omics_clinical)[0]/10]]

            #devide between healthy and sick
            X_train, X_test, y_train, y_test = train_test_split(omics_clinical, clinical['group'], test_size=0.3, random_state=42)

            #KNN imputer(default parameters): train on train data, apply on both train and test
            imp = KNNImputer(n_neighbors=5, weights="uniform")
            imp.fit(X_train)
            X_train = pd.DataFrame(imp.transform(X_train), index = X_train.index, columns = X_train.columns)
            X_test = pd.DataFrame(imp.transform(X_test), index = X_test.index, columns = X_test.columns)



            # ExtraTreesClassifier(default parameters): Choose top 5 proteins/metabolites
            clf = ExtraTreesClassifier(n_estimators=10000, random_state=42, verbose = 1).fit(X_train, y_train)


            for N in range(1,31):
            #for N in range(4,6):
                # Take only top N proteins/metabolites
                model = SelectFromModel(clf, prefit=True, max_features = N)
                X_train_subset = X_train[X_train.columns[model.get_support()]]
                X_test_subset = X_test[X_test.columns[model.get_support()]]


                #train ridge regression model

                clf_ridge = LogisticRegressionCV(cv=10, Cs = 10, random_state=42, max_iter = 10000,
                                           penalty='l2').fit(X_train_subset, y_train)



                #compute feature importance 
                coefficients = clf_ridge.coef_[0]
                feature_importance = pd.DataFrame({'Feature': X_train_subset.columns,
                                                   'Importance': np.abs(coefficients),
                                                   'Coef': coefficients})
                feature_importance = feature_importance.sort_values('Importance', ascending = True)


                #compute metrics
                auc_train = roc_auc_score(y_train, clf_ridge.predict_proba(X_train_subset)[:, 1])
                auc = roc_auc_score(y_test, clf_ridge.predict_proba(X_test_subset)[:, 1])
                conf_matrix = confusion_matrix(y_test, clf_ridge.predict(X_test_subset), labels = np.unique(y_test))


                #print summary
                print('Disease: ' + disease)
                print(conf_matrix)
                print('AUC train: ' + str(auc_train))
                print('AUC: ' + str(auc))
                print('\n')



             
                # Prediction for test dataset
                X_test_summary = pd.DataFrame(clf_ridge.predict_proba(X_test_subset)[:, 1], columns = ['prob'])
                X_test_summary['reference'] = y_test.values
                

                # Data for boxplot
                if N == 5:      
                    boxplot_data = X_test_summary[['prob', 'reference']]
                    boxplot_data['disease'] = disease
                    boxplot_data['data_type'] = data_type
                    boxplot_data['Model'] = model_type
                    boxplot_data_full = boxplot_data_full.append(boxplot_data, ignore_index=True)

                # Data for ShinyApp
                new_row = {'prob': list(np.round(boxplot_data['prob'].astype(float), 3)),
                           'reference': list(boxplot_data['reference']),
                           'Disease': disease,
                           'Data': data_type,
                           'N': N,
                           'AUC_train': auc_train,
                           'AUC_test': auc,
                           'Features': list(feature_importance['Feature'].astype(str)),
                           'Coef': list(np.round(feature_importance['Coef'], 3)),
                           'Model': model_type
                    }

                Shiny_data = Shiny_data.append(new_row, ignore_index=True)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 10000 out of 10000 | elapsed:   14.7s finished


Disease: CD
[[37 18]
 [28 29]]
AUC train: 0.6841529084561216
AUC: 0.625518341307815


Disease: CD
[[39 16]
 [28 29]]
AUC train: 0.7031777028226524
AUC: 0.6389154704944179


Disease: CD
[[36 19]
 [26 31]]
AUC train: 0.7456062488904668
AUC: 0.6277511961722488


Disease: CD
[[36 19]
 [26 31]]
AUC train: 0.7637730043197822
AUC: 0.6411483253588517


Disease: CD
[[35 20]
 [24 33]]
AUC train: 0.7725900940884075
AUC: 0.6615629984051037


Disease: CD
[[35 20]
 [26 31]]
AUC train: 0.7656074323924493
AUC: 0.6446570972886763


Disease: CD
[[35 20]
 [26 31]]
AUC train: 0.7654890821942126
AUC: 0.65103668261563


Disease: CD
[[40 15]
 [28 29]]
AUC train: 0.7632996035268358
AUC: 0.6465709728867624


Disease: CD
[[38 17]
 [27 30]]
AUC train: 0.7635363039233092
AUC: 0.6376395534290271


Disease: CD
[[40 15]
 [26 31]]
AUC train: 0.7667317592756968
AUC: 0.6401913875598086


Disease: CD
[[39 16]
 [23 34]]
AUC train: 0.7890407716432926
AUC: 0.7196172248803828


Disease: CD
[[40 15]
 [29 28]]
AUC train: 0.76

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 10000 out of 10000 | elapsed:   13.7s finished


Disease: CD
[[25 23]
 [19 33]]
AUC train: 0.7312295569432055
AUC: 0.5773237179487181


Disease: CD
[[30 18]
 [18 34]]
AUC train: 0.7667261373773416
AUC: 0.6638621794871795


Disease: CD
[[31 17]
 [16 36]]
AUC train: 0.8029289325007434
AUC: 0.6979166666666666


Disease: CD
[[33 15]
 [17 35]]
AUC train: 0.8174249182277729
AUC: 0.6919070512820512


Disease: CD
[[33 15]
 [16 36]]
AUC train: 0.8230002973535533
AUC: 0.703926282051282


Disease: CD
[[35 13]
 [17 35]]
AUC train: 0.8374962830805827
AUC: 0.7123397435897435


Disease: CD
[[33 15]
 [16 36]]
AUC train: 0.8539250669045495
AUC: 0.7467948717948718


Disease: CD
[[43  5]
 [19 33]]
AUC train: 0.8710972346119537
AUC: 0.8149038461538461


Disease: CD
[[43  5]
 [17 35]]
AUC train: 0.8745168004757656
AUC: 0.8165064102564102


Disease: CD
[[40  8]
 [18 34]]
AUC train: 0.8887154326494202
AUC: 0.8277243589743589


Disease: CD
[[40  8]
 [14 38]]
AUC train: 0.8898305084745763
AUC: 0.8377403846153846


Disease: CD
[[38 10]
 [14 38]]
AUC train: 0.

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 10000 out of 10000 | elapsed:   37.5s finished


...

Disease: Atherosclerosis
[[23  2]
 [13 13]]
AUC train: 0.831385154880187
AUC: 0.6907692307692307


Disease: Atherosclerosis
[[23  2]
 [12 14]]
AUC train: 0.847749853886616
AUC: 0.7846153846153846


Disease: Atherosclerosis
[[23  2]
 [ 7 19]]
AUC train: 0.880771478667446
AUC: 0.883076923076923


Disease: Atherosclerosis
[[24  1]
 [ 5 21]]
AUC train: 0.9032729398012858
AUC: 0.8923076923076924


Disease: Atherosclerosis
[[23  2]
 [ 5 21]]
AUC train: 0.9032729398012858
AUC: 0.8923076923076924


Disease: Atherosclerosis
[[24  1]
 [ 5 21]]
AUC train: 0.9102863822326125
AUC: 0.9030769230769231


Disease: Atherosclerosis
[[22  3]
 [ 3 23]]
AUC train: 0.923728813559322
AUC: 0.9323076923076923


Disease: Atherosclerosis
[[22  3]
 [ 3 23]]
AUC train: 0.9237288135593221
AUC: 0.9307692307692308


Disease: Atherosclerosis
[[22  3]
 [ 3 23]]
AUC train: 0.924021040327294
AUC: 0.9307692307692308


Disease: Atherosclerosis
[[22  3]
 [ 3 23]]
AUC train: 0.9304500292226767
AUC: 0.9246153846153845


Diseas

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 10000 out of 10000 | elapsed:  1.1min finished


Disease: Atherosclerosis
[[392 188]
 [298 313]]
AUC train: 0.6461690975529286
AUC: 0.6391020937976184


Disease: Atherosclerosis
[[380 200]
 [249 362]]
AUC train: 0.687322974832079
AUC: 0.6652040182854564


Disease: Atherosclerosis
[[395 185]
 [228 383]]
AUC train: 0.7242962049954038
AUC: 0.710720130932897


Disease: Atherosclerosis
[[403 177]
 [232 379]]
AUC train: 0.7276354313863418
AUC: 0.710900728032056


Disease: Atherosclerosis
[[402 178]
 [226 385]]
AUC train: 0.7293182728361169
AUC: 0.7134601275467013


Disease: Atherosclerosis
[[403 177]
 [227 384]]
AUC train: 0.7293551044461459
AUC: 0.7130678932219652


Disease: Atherosclerosis
[[407 173]
 [226 385]]
AUC train: 0.7333033492902392
AUC: 0.7195637451323438


Disease: Atherosclerosis
[[410 170]
 [233 378]]
AUC train: 0.7355324398295993
AUC: 0.722552062757492


Disease: Atherosclerosis
[[410 170]
 [230 381]]
AUC train: 0.7355827590714699
AUC: 0.7228822168293922


Disease: Atherosclerosis
[[407 173]
 [222 389]]
AUC train: 0.7384711

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 10000 out of 10000 | elapsed:   12.8s finished


Disease: Atherosclerosis
[[72 19]
 [36 64]]
AUC train: 0.7393254761225071
AUC: 0.7678021978021978


Disease: Atherosclerosis
[[73 18]
 [38 62]]
AUC train: 0.7481750336446312
AUC: 0.7487912087912089


Disease: Atherosclerosis
[[75 16]
 [37 63]]
AUC train: 0.7608172586762367
AUC: 0.7895604395604396


Disease: Atherosclerosis
[[75 16]
 [38 62]]
AUC train: 0.763998205619673
AUC: 0.7993406593406593


Disease: Atherosclerosis
[[77 14]
 [40 60]]
AUC train: 0.7761714448839769
AUC: 0.8042857142857143


Disease: Atherosclerosis
[[77 14]
 [43 57]]
AUC train: 0.7903837527017658
AUC: 0.799120879120879


Disease: Atherosclerosis
[[76 15]
 [42 58]]
AUC train: 0.7965417397332899
AUC: 0.8067032967032967


Disease: Atherosclerosis
[[77 14]
 [40 60]]
AUC train: 0.8112230333183801
AUC: 0.8283516483516482


Disease: Atherosclerosis
[[77 14]
 [39 61]]
AUC train: 0.810183108356103
AUC: 0.8204395604395605


Disease: Atherosclerosis
[[77 14]
 [39 61]]
AUC train: 0.8101423269850332
AUC: 0.8206593406593407


Dis

In [ ]:
Shiny_data = Shiny_data[['Disease', 'Model', 'Data', 'N', 'AUC_train', 'AUC_test', 'Features', 'Coef', 'prob', 'reference']]

In [46]:
Shiny_data

,prob,reference,Disease,Data,N,AUC_train,AUC_test,Features,Coef,Model
0,"[0.457, 0.488, 0.474, 0.431, 0.571, 0.554, 0.4...","[HC, Incident, Incident, HC, Incident, HC, Inc...",CD,prot,1.0,0.684153,0.625518,[CXCL9],[0.54],incident
1,"[0.469, 0.504, 0.49, 0.472, 0.519, 0.512, 0.49...","[HC, Incident, Incident, HC, Incident, HC, Inc...",CD,prot,2.0,0.703178,0.638915,"[CRHBP, CXCL9]","[0.079, 0.179]",incident
2,"[0.238, 0.659, 0.725, 0.575, 0.628, 0.66, 0.52...","[HC, Incident, Incident, HC, Incident, HC, Inc...",CD,prot,3.0,0.745606,0.627751,"[CXCL9, MMP10, CRHBP]","[0.527, 0.564, 1.045]",incident
3,"[0.327, 0.726, 0.662, 0.673, 0.67, 0.734, 0.60...","[HC, Incident, Incident, HC, Incident, HC, Inc...",CD,prot,4.0,0.763773,0.641148,"[CXCL9, CHI3L1, MMP10, CRHBP]","[0.414, 0.486, 0.605, 1.209]",incident
4,"[0.488, 0.697, 0.695, 0.656, 0.767, 0.85, 0.67...","[HC, Incident, Incident, HC, Incident, HC, Inc...",CD,prot,5.0,0.772590,0.661563,"[CXCL9, KRT19, CHI3L1, MMP10, CRHBP]","[0.359, 0.425, 0.462, 0.521, 0.743]",incident
...,...,...,...,...,...,...,...,...,...,...
1075,"[0.349, 0.665, 0.672, 0.324, 0.547, 0.144, 0.3...","[HC, Prevalent, Prevalent, HC, HC, HC, HC, Pre...",Atherosclerosis,met,26.0,0.804678,0.823187,"[L_LDL_PL, Total_C, L_LDL_L, XS_VLDL_FC_by_CE,...","[-0.006, 0.007, 0.007, -0.013, 0.013, 0.022, 0...",prevalent
1076,"[0.348, 0.665, 0.673, 0.323, 0.548, 0.144, 0.3...","[HC, Prevalent, Prevalent, HC, HC, HC, HC, Pre...",Atherosclerosis,met,27.0,0.804922,0.823297,"[L_LDL_PL, XS_VLDL_FC_by_CE, L_LDL_L, L_LDL_FC...","[-0.005, -0.013, 0.014, 0.015, 0.026, 0.026, 0...",prevalent
1077,"[0.354, 0.676, 0.662, 0.322, 0.541, 0.147, 0.3...","[HC, Prevalent, Prevalent, HC, HC, HC, HC, Pre...",Atherosclerosis,met,28.0,0.805983,0.823956,"[L_LDL_PL, XS_VLDL_FC_by_CE, L_LDL_FC, LDL_C, ...","[0.003, -0.013, 0.02, -0.024, 0.027, 0.034, 0....",prevalent
1078,"[0.354, 0.678, 0.664, 0.322, 0.542, 0.147, 0.3...","[HC, Prevalent, Prevalent, HC, HC, HC, HC, Pre...",Atherosclerosis,met,29.0,0.805942,0.823956,"[L_LDL_PL, XS_VLDL_FC_by_CE, L_LDL_FC, LDL_C, ...","[0.003, -0.013, 0.02, -0.025, 0.03, 0.034, 0.0...",prevalent


In [47]:
boxplot_data_full

,prob,reference,disease,data_type,Model
0,0.488438,HC,CD,prot,incident
1,0.696740,Incident,CD,prot,incident
2,0.694683,Incident,CD,prot,incident
3,0.655752,HC,CD,prot,incident
4,0.766940,Incident,CD,prot,incident
...,...,...,...,...,...
54816,0.462279,Prevalent,Atherosclerosis,met,prevalent
54817,0.610119,Prevalent,Atherosclerosis,met,prevalent
54818,0.254494,HC,Atherosclerosis,met,prevalent
54819,0.592955,Prevalent,Atherosclerosis,met,prevalent


In [49]:
Shiny_data.to_csv('ShinyData_clinical_story.csv') #Supplementary table 2 (proteomics + metabolomics)

In [51]:
boxplot_data_full.to_csv('boxplot_data_clinical_story.csv')